In [2]:
import pandas as pd
import numpy as np
import random
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# # Establecemos una semilla para reducir la aleatoriedad de los datos
# np.random.seed(123)
# random.seed(123)

data_basepath = '..\\data\\spain_dataframes\\'

In [3]:
# Leer el fichero con los datos
df = pd.read_csv(f'{data_basepath}datos_tratados.csv')
df

,comunidad_autonoma,edad,genero,actividad_fisica,asistencia_cine,asistencia_directos,asistencia_cultural,asistencia_deporte,dias_alta_contaminacion,homicidios_100mhabit,criminalidad_1000habit,estudios,estado_civil,horasTrabajadas_mes,salario_anual,satisf_hospitales,satisf_dentistas,satisf_especialistas,satisf_medGeneral
0,Extremadura,13,mujer,nivel_alto,no,no,no,no,0.0000,0.3842,32.7322,educacion_superior,soltero/a,85.6267,15049.6479,hospitales_insatisfecho/a,dentistas_satisfecho/a,especialistas_neutral,medGeneral_insatisfecho/a
1,Extremadura,19,hombre,nivel_moderado,no,si,no,no,0.3615,0.5224,41.5967,primaria,casado/a,197.2631,26898.1759,hospitales_insatisfecho/a,dentistas_satisfecho/a,especialistas_satisfecho/a,medGeneral_neutral
2,Extremadura,41,hombre,nivel_alto,no,no,no,si,5.2239,0.4281,25.3917,primero_secundaria,casado/a,166.4791,23817.8688,hospitales_muy_satisfecho/a,dentistas_satisfecho/a,especialistas_insatisfecho/a,medGeneral_neutral
3,Extremadura,15,hombre,nivel_alto,no,no,no,no,4.4288,0.0000,34.4999,segundo_secundaria_profesional_,divorciado/a,109.1762,26785.9832,hospitales_satisfecho/a,dentistas_satisfecho/a,especialistas_satisfecho/a,medGeneral_satisfecho/a
4,Extremadura,77,hombre,nivel_alto,si,no,si,no,0.6225,0.3840,29.8457,primero_secundaria,soltero/a,170.2207,19347.2427,hospitales_insatisfecho/a,dentistas_satisfecho/a,especialistas_neutral,medGeneral_satisfecho/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Melilla,57,mujer,nivel_alto,no,si,no,no,15.3718,0.2086,54.6253,primero_secundaria,casado/a,157.2102,24160.0415,hospitales_muy_insatisfecho/a,dentistas_satisfecho/a,especialistas_neutral,medGeneral_insatisfecho/a
1996,Melilla,35,hombre,nivel_moderado,si,no,no_puede,si,12.3848,0.3044,56.6916,segundo_secundaria_general,casado/a,170.8738,25288.0479,hospitales_satisfecho/a,dentistas_satisfecho/a,especialistas_satisfecho/a,medGeneral_satisfecho/a
1997,Melilla,82,mujer,nivel_moderado,si,no,si,no,15.0574,0.1649,45.9722,primero_secundaria,soltero/a,147.9526,25514.6306,hospitales_insatisfecho/a,dentistas_satisfecho/a,especialistas_satisfecho/a,medGeneral_satisfecho/a
1998,Melilla,35,hombre,nivel_alto,si,no_puede,si,no,12.7647,0.0000,39.4826,educacion_superior,soltero/a,143.9239,19769.7060,hospitales_muy_insatisfecho/a,dentistas_satisfecho/a,especialistas_neutral,medGeneral_muy_satisfecho/a


### Homicidios y Criminalidad --> Percepción de seguridad

Se establece una nota media de la percepción de la seguridad para cada individuo, en función de la media de las notas de los homicidios y la criminalidad.

In [4]:
# Valoración de una columna dentro de un rango determinado
def valorate_numeric_column(df, column_name, new_column_name, min_rank_value=0, max_rank_value=1, revert_rank_valoration=False):
    """
    Normaliza los valores de una columna de un DataFrame dentro de un rango específico y crea una nueva columna con los resultados.

    La valoración se realiza de forma lineal, permitiendo definir un rango mínimo y máximo.
    Se toman como valores extremos los valores mínimo y máximo de la columna especificada en 'column_name'.

    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame que contiene la columna a normalizar.
    column_name : str
        Nombre de la columna que se va a normalizar.
    new_column_name : str
        Nombre de la nueva columna donde se almacenarán los valores normalizados.
    min_rank_value : float, opcional (por defecto=0)
        Valor mínimo del rango.
    max_rank_value : float, opcional (por defecto=1)
        Valor máximo del rango.
    revert_rank_valoration : bool, opcional (por defecto=False)
        Si es True, invierte la valoración (el valor más alto se convierte en el más bajo y viceversa).

    Retorna:
    --------
    None. Modifica el DataFrame original añadiendo una nueva columna con los valores normalizados.
    """
    # Valores extremos del dataframe
    min_val = df[column_name].min()
    max_val = df[column_name].max()

    # Proporcionalidad entre el rango definido y el rango calculado
    rank_diference = max_rank_value - min_rank_value

    # Aplicar la fórmula de normalización lineal
    if not revert_rank_valoration:
        df[new_column_name] = ((df[column_name] - min_val) / (max_val - min_val)) * rank_diference + min_rank_value
    else:
        df[new_column_name] = (1 - ((df[column_name] - min_val) / (max_val - min_val))) * rank_diference + min_rank_value

In [5]:
# Valoración de la criminalidad
valorate_numeric_column(df, 'criminalidad_1000habit', 'valoracion_criminalidad', 0, 10, True)

# Valoración de la tasa de homicidios
valorate_numeric_column(df, 'homicidios_100mhabit', 'valoracion_homicidios', 0, 10, True)

df[['criminalidad_1000habit', 'valoracion_criminalidad', 'homicidios_100mhabit', 'valoracion_homicidios']].head()

,criminalidad_1000habit,valoracion_criminalidad,homicidios_100mhabit,valoracion_homicidios
0,32.7322,6.386014,0.3842,9.369523
1,41.5967,5.407278,0.5224,9.142735
2,25.3917,7.196484,0.4281,9.297483
3,34.4999,6.190841,0.0000,10.000000
4,29.8457,6.704714,0.3840,9.369851


In [6]:
# Hacer una valoración media "percepcion_seguridad" entre la criminalidad y la tasa de homicidios
df['percepcion_seguridad'] = (df['valoracion_criminalidad'] + df['valoracion_homicidios']) / 2
df['percepcion_seguridad']

0       7.877768
1       7.275006
2       8.246983
3       8.095420
4       8.037283
          ...   
1995    6.813233
1996    6.620557
1997    7.326786
1998    7.820348
1999    2.447994
Name: percepcion_seguridad, Length: 2000, dtype: float64

### Estado civil

Se limpian los valores de estado civil, siendo soltero/a para los menores de 18 años, y el resto de valores se mantienen.

In [7]:
df.loc[df['edad'] < 18, 'estado_civil'] = 'soltero/a'
df[df['edad'] < 18]['estado_civil'].unique()

array(['soltero/a'], dtype=object)

### Nivel de estudios

Se limpian los valores de estudios que por edad no son coherentes con el máximo nivel de estudios alcanzado.

In [8]:
df['estudios'].unique().tolist()

['educacion_superior',
 'primaria',
 'primero_secundaria',
 'segundo_secundaria_profesional_',
 'segundo_secundaria_general',
 'analfabetos',
 'estudios_primarios_incompletos']

In [9]:
# Se ordenan los niveles de educación de mayor a menor nivel
ordered_education = ['educacion_superior',
                     'segundo_secundaria_profesional_',
                     'segundo_secundaria_general',
                     'primero_secundaria',
                     'primaria',
                     'estudios_primarios_incompletos',
                     'analfabetos']

In [10]:
# Se recorre todo el dataframe para comprobar registro por registro el nivel de estudios alcanzado por cada persona en función de su edad.
iterator = 0 # Iterador entre profesional y general en secundaria
for index, row in df.iterrows():
    # Comprobamos el nivel de estudios máximo al que puede obtar por su edad

    if row['edad'] < 10 and ordered_education.index(row['estudios']) < 5:
        # Si es menor de 10, los máximos estudios alcanzados son estudios primarios incompletos
        df.loc[index, 'estudios'] = ordered_education[5]

    if row['edad'] < 12 and ordered_education.index(row['estudios']) < 4:
        # Si es menor de 12, los máximos estudios alcanzados son primaria
        df.loc[index, 'estudios'] = ordered_education[4]

    elif row['edad'] < 16 and ordered_education.index(row['estudios']) < 3:
        # Si es menor de 16, los máximos estudios alcanzados son primero de secundaria
        df.loc[index, 'estudios'] = ordered_education[3]

    elif row['edad'] < 18 and ordered_education.index(row['estudios']) < 2:
        # Si es menor de 18, los máximos estudios alcanzados son segundo de secundaria
        df.loc[index, 'estudios'] = ordered_education[1 + iterator]
        iterator = 1 if iterator == 0 else 0 # Cambia entre profesional y general para la siguiente iteración

In [11]:
df[df['edad'] < 23]['estudios'].unique()

array(['primero_secundaria', 'primaria',
       'segundo_secundaria_profesional_', 'segundo_secundaria_general',
       'educacion_superior', 'estudios_primarios_incompletos',
       'analfabetos'], dtype=object)

In [23]:
df.loc[df['edad'] < 16, 'salario_anual'] = 0

In [25]:
df.loc[df['edad'] > 75, 'actividad_fisica'] = 'nivel_bajo'